In [ ]:
import os
import time
import config
from datetime import datetime
import ollama
from codecarbon import EmissionsTracker, OfflineEmissionsTracker
from log_utils import read_log_messages, normalize_template, normalize_template_v1, normalize_template_v2, save_templates
from debt_utils import get_code_snippets, get_td_ground_truth, save_td_labels, normalize_td_label
from ollama_utils import start_ollama_server, stop_ollama_server, start_ollama_server_log
from evaluation import evaluate_and_save_parsing, evaluate_and_save_td

In [ ]:
# --- Configuration ---
llm_config = config.LLM_CONFIG

DATA_DIR = config.DATA_DIR
RESULT_DIR = config.RESULT_DIR
os.makedirs(RESULT_DIR, exist_ok=True)
 
TASK = (config.TASK).capitalize()
DESIGN = (config.DESIGN).capitalize()
project_name = f"{TASK}_{DESIGN}"
model_name = llm_config["config_list"][0]["model"]
model = llm_config["config_list"][0]["model"].replace(":", "-")
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
exp_name = f"{project_name}_{model}_{timestamp}"

input_file = config.IN_FILE
input_file_path = os.path.join(DATA_DIR, input_file)
ground_truth_file = config.GT_FILE
ground_truth_file_path = os.path.join(DATA_DIR, ground_truth_file)



In [ ]:
# --- Ollama Query Function ---
def ask_ollama(model, prompt, temperature=0):
    try:
        response = ollama.generate(
            model=model,
            prompt=prompt,
            options={'temperature': temperature},
            think = False
        )
    except ollama.ResponseError as e:
        print('Error:', e.error)
        return None
    return response.get('response', None)

TD Detection

In [ ]:
# --- Task-specific configuration ---
task_prompt = config.TASK_PROMPT_TD_DETECTION
sys_prompt_generator = config.SYS_MSG_TD_DETECTION_GENERATOR_FEW_SHOT

In [ ]:
code_snippets = get_code_snippets(input_file_path)

In [ ]:
# --- TD Detection With CodeCarbon ---
def run_inference_with_emissions_td_detection(code_snippets, model_name, sys_prompt, exp_name, result_dir):
    td_results = []
    tracker = OfflineEmissionsTracker(project_name=exp_name, output_dir=result_dir, country_iso_code="CAN", save_to_file=True)
    tracker.start()
    try:
        for i, code_snippet in enumerate(code_snippets):
            #print(f"Processing code_snippet {i+1}/{len(code_snippets)}")
            prompt = sys_prompt + code_snippet
            response = ask_ollama(model_name, prompt)
            if response is not None:
                td_results.append(response)
                
            else:
                print(f"[Warning] Skipped code snippet {i} — no response or invalid format.")
    finally:
        emissions = tracker.stop()
    print(f"Emissions: {emissions} kg CO2")
    return td_results


In [ ]:
proc = start_ollama_server()
time.sleep(5) 
td_results = run_inference_with_emissions_td_detection(code_snippets, model_name, sys_prompt_generator, exp_name, RESULT_DIR)
save_td_labels(td_results, llm_config, DESIGN, RESULT_DIR)
stop_ollama_server(proc)

In [ ]:
# Load ground truth
gt = get_td_ground_truth(ground_truth_file_path)
# Evaluate and save results
results = evaluate_and_save_td(normalize_td_label, gt, td_results, exp_name)

Log Parsing

In [ ]:
# --- Task-specific configuration ---
task_prompt = config.TASK_PROMPT_LOG_PARSING
sys_prompt_generator = config.SYS_MSG_SINGLE_LOG_PARSER_FEW_SHOT

In [ ]:
# --- Read Input Data ---
logs = read_log_messages(input_file_path)

In [ ]:
# --- Log Parsing With CodeCarbon ---
def run_inference_with_emissions_log_parsing(logs, model_name, prompt_prefix, task, exp_name, result_dir):
    parsed_templates = []
    tracker = EmissionsTracker(project_name=exp_name, output_dir=result_dir, save_to_file=True)
    tracker.start()
    try:
        for i, log_message in enumerate(logs):
            print(f"Processing log {i+1}/{len(logs)}")
            prompt = prompt_prefix + task + log_message
            response = ask_ollama(model_name, prompt)
            if response is not None:
                parsed_templates.append(response)
                print(response)
            else:
                print(f"[Warning] Skipped log {i} — no response or invalid format.")
    finally:
        emissions = tracker.stop()
    print(f"Emissions: {emissions} kg CO2")
    return parsed_templates


In [ ]:
proc = start_ollama_server()
time.sleep(5) # Give it some time to initialize
parsed_templates = run_inference_with_emissions_log_parsing(logs, model_name, sys_prompt_generator, task_prompt, exp_name, RESULT_DIR)
save_templates(parsed_templates, llm_config, DESIGN, RESULT_DIR)
stop_ollama_server(proc)

In [ ]:
# using normalize_template
results = evaluate_and_save_parsing(normalize_template, parsed_templates, ground_truth_file_path, exp_name)
# using normalize_template_v1
results_v1 = evaluate_and_save_parsing(normalize_template_v1, parsed_templates, ground_truth_file_path, exp_name)
# using normalize_template_v2
results_v2 = evaluate_and_save_parsing(normalize_template_v2, parsed_templates, ground_truth_file_path, exp_name)

In [ ]:
#print("Results:", results)